In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\mites\AppData\Local\Temp\ipykernel_18016\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [5]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\mites\Desktop\Ironhack\Week7\Day1lab\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [9]:
# Your code
from sklearn.model_selection import train_test_split

# Split into train (80%) and validation (20%)
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

print(data_train.shape, data_val.shape)


(800, 2) (200, 2)


## Data Preprocessing

In [10]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [11]:
# Your code
import re

def clean_html(text):
    # Remove inline JavaScript/CSS
    text = re.sub(r"<script.*?>.*?</script>", "", text, flags=re.S)

    # Remove HTML comments
    text = re.sub(r"<!--.*?-->", "", text, flags=re.S)

    # Remove remaining HTML tags
    text = re.sub(r"<.*?>", "", text)

    return text

data_train["clean_text"] = data_train["text"].apply(clean_html)
data_val["clean_text"] = data_val["text"].apply(clean_html)


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [12]:
# Your code
def basic_cleaning(text):

    # Remove special characters
    text = re.sub(r"[^A-Za-z\s]", " ", text)

    # Remove numbers
    text = re.sub(r"\d+", " ", text)

    # Remove single characters
    text = re.sub(r"\b\w\b", " ", text)

    # Remove single characters at the start
    text = re.sub(r"^\w\s+", "", text)

    # Replace multiple spaces with single
    text = re.sub(r"\s+", " ", text)

    # Lowercase
    text = text.lower()

    return text

data_train["clean_text"] = data_train["clean_text"].apply(basic_cleaning)
data_val["clean_text"] = data_val["clean_text"].apply(basic_cleaning)


## Now let's work on removing stopwords
Remove the stopwords.

In [13]:
# Your code
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    words = text.split()
    filtered = [w for w in words if w not in stop_words]
    return " ".join(filtered)

data_train["clean_text"] = data_train["clean_text"].apply(remove_stopwords)
data_val["clean_text"] = data_val["clean_text"].apply(remove_stopwords)


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [14]:
# Your code
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemm.lemmatize(w) for w in text.split()])

data_train["preprocessed_text"] = data_train["clean_text"].apply(lemmatize_text)
data_val["preprocessed_text"] = data_val["clean_text"].apply(lemmatize_text)


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [15]:
# Your code
from collections import Counter

# Separate ham and spam
ham_words = " ".join(data_train[data_train.label=="ham"]["preprocessed_text"]).split()
spam_words = " ".join(data_train[data_train.label=="spam"]["preprocessed_text"]).split()

print("Top HAM words:", Counter(ham_words).most_common(10))
print("Top SPAM words:", Counter(spam_words).most_common(10))


Top HAM words: []
Top SPAM words: []


## Extra features

In [16]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,clean_text,preprocessed_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,regards mr nelson smith kindly reply private e...,regard mr nelson smith kindly reply private em...,0,0,79
535,I have not been able to reach oscar this am. W...,0,able reach oscar supposed send pdb receive,able reach oscar supposed send pdb receive,0,0,42
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,huma abedin checking pat work jack jake rest a...,huma abedin checking pat work jack jake rest a...,0,0,76
557,I can have it announced here on Monday - can't...,0,announced monday today,announced monday today,0,0,22
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,bank africaagence san pedro bp san pedro cote ...,bank africaagence san pedro bp san pedro cote ...,1,1,1050


## How would work the Bag of Words with Count Vectorizer concept?

In [18]:
# Your code

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X_train_bow = cv.fit_transform(data_train["preprocessed_text"])
X_val_bow = cv.transform(data_val["preprocessed_text"])

print(X_train_bow.shape)


(800, 28166)


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [19]:
# Your code
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(data_train["preprocessed_text"])
X_val_tfidf = tfidf.transform(data_val["preprocessed_text"])

print(X_train_tfidf.shape)


(800, 28166)


## And the Train a Classifier?

In [20]:
# Your code

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB()

clf.fit(X_train_tfidf, data_train["label"])
pred = clf.predict(X_val_tfidf)

print("Accuracy:", accuracy_score(data_val["label"], pred))


Accuracy: 0.935


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code

